In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")


In [2]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
from src.database import DatabaseManager, SQLQueries

In [3]:
db = DatabaseManager()
db.conn = db.conn or db.create_database(csv_folder=str(Path.cwd().parent / "data"))

Creating database at: outputs/nonprofit_grants.db

Loading grants.csv...

Table 'grants': 75,640 rows, 22 columns
Loading grants_final.csv...

Table 'grants_final': 75,640 rows, 24 columns
Loading non-profits.csv...

Table 'non_profits': 240,585 rows, 28 columns
Loading non-profits_final.csv...

Table 'non_profits_final': 240,585 rows, 33 columns
Loading nonprofit_anomalies.csv...

Table 'nonprofit_anomalies': 240,585 rows, 9 columns
Loading nonprofit_quality.csv...

Table 'nonprofit_quality': 240,585 rows, 9 columns

Database created successfully!



In [4]:
tables = ['grants', 'grants_final', 'non_profits', 'non_profits_final',
              'nonprofit_anomalies', 'nonprofit_quality']

print("\nTable Metadata:")
print("-" * 70)
    
for table in tables:
        try:
            info = db.get_table_info(table)
            print(f"\n{table}:")
            print(f"  Rows: {info['row_count']:,}")
            print(f"  Columns: {len(info['columns'])}")
            print(f"  Columns: {', '.join(info['sample_data'].columns[:].tolist())}...")
        except Exception as e:
            print(f"\n{table}: Error - {str(e)}")


Table Metadata:
----------------------------------------------------------------------

grants:
  Rows: 75,640
  Columns: 22
  Columns: Unnamed: 0, opportunity_id, opportunity_title, opportunity_number, opportunity_category, funding_instrument_type, category_of_funding_activity, cfda_numbers, eligible_applicants, eligible_applicants_type, agency_code, agency_name, post_date, close_date, last_updated_date, archive_date, award_ceiling, award_floor, estimated_total_program_funding, expected_number_of_awards, cost_sharing_or_matching_requirement, additional_information_url...

grants_final:
  Rows: 75,640
  Columns: 24
  Columns: Unnamed: 0, opportunity_id, opportunity_title, opportunity_number, opportunity_category, funding_instrument_type, category_of_funding_activity, cfda_numbers, eligible_applicants, eligible_applicants_type, agency_code, agency_name, post_date, close_date, last_updated_date, archive_date, award_ceiling, award_floor, estimated_total_program_funding, expected_number_o

### Grant Analysis


In [5]:
print("\nTOP GRANTS BY FUNDING AMOUNT")

query_top_grants = """
SELECT 
    opportunity_title,
    agency_name,
    award_ceiling,
    award_floor,
    estimated_total_program_funding,
    close_date
FROM grants
WHERE award_ceiling IS NOT NULL
ORDER BY award_ceiling DESC
LIMIT 15
"""

top_grants = db.execute_query(query_top_grants)
top_grants




TOP GRANTS BY FUNDING AMOUNT


,opportunity_title,agency_name,award_ceiling,award_floor,estimated_total_program_funding,close_date
0,National Clean Investment Fund (NCIF),Environmental Protection Agency,1.397000e+10,NaN,1.397000e+10,2023-10-12
1,Commercial Technologies for Maintenance Activi...,Washington Headquarters Services,1.000000e+10,0.0,NaN,2023-11-06
2,"Bridge Investment Program - Planning, Bridge P...",DOT Federal Highway Administration,1.000000e+10,2500000.0,NaN,2022-09-08
3,FY22-23 Federal-State Partnership for Intercit...,DOT - Federal Railroad Administration,8.979150e+09,0.0,8.979150e+09,2023-03-27
4,Clean Communities Investment Accelerator (CCIA),Environmental Protection Agency,6.000000e+09,NaN,6.000000e+09,2023-10-12
5,FY22 Federal-State Partnership for Intercity P...,DOT - Federal Railroad Administration,4.566300e+09,0.0,NaN,2023-04-21
6,Advanced Reactor Demonstration,Idaho Field Office,4.000000e+09,0.0,NaN,2020-08-19
7,FY23 National Network IIJA Supplemental Cooper...,DOT - Federal Railroad Administration,3.143000e+09,0.0,3.143000e+09,2023-06-15
8,National Network IIJA Supplemental Cooperative...,DOT - Federal Railroad Administration,3.113000e+09,0.0,3.113000e+09,2022-09-21
9,Fiscal Year (FY) 2023 &#8211; 2026 Bridge Inve...,DOT Federal Highway Administration,3.000000e+09,50000000.0,9.620100e+09,2024-08-01


In [6]:
# Visualization
fig = px.bar(
    top_grants, 
    x='award_ceiling', 
    y='opportunity_title',
    orientation='h',
    title='Top 15 Grants by Award Ceiling',
    labels={'award_ceiling': 'Award Ceiling ($)', 'opportunity_title': 'Grant Title'},
    color='award_ceiling',
    color_continuous_scale='Blues'
)
fig.update_layout(height=600, showlegend=True)
fig.show()

In [7]:
print("GRANT DISTRIBUTION BY AGENCY")


query_grants_by_agency = """
SELECT 
    agency_name,
    COUNT(*) as grant_count,
    AVG(award_ceiling) as avg_award,
    SUM(estimated_total_program_funding) as total_funding
FROM grants
WHERE agency_name IS NOT NULL
GROUP BY agency_name
ORDER BY grant_count DESC
LIMIT 20
"""

grants_by_agency = db.execute_query(query_grants_by_agency)
grants_by_agency

GRANT DISTRIBUTION BY AGENCY


,agency_name,grant_count,avg_award,total_funding
0,National Institutes of Health,11423,7.550015e+05,1.938767e+10
1,National Park Service,8438,1.605700e+05,5.074960e+09
2,Fish and Wildlife Service,4148,8.504687e+05,6.840445e+09
3,Bureau of Land Management,2929,4.095641e+05,2.538203e+09
4,Geological Survey,2332,3.788617e+05,8.054034e+08
5,Centers for Disease Control and Prevention,1728,1.838765e+06,3.090953e+10
6,Health Resources and Services Administration,1610,4.013532e+05,5.589612e+10
7,Department of Education,1523,2.932490e+06,1.273578e+11
8,Dept. of the Army -- USAMRAA,1507,1.064872e+06,1.553294e+10
9,Environmental Protection Agency,1388,1.842835e+07,4.250399e+10


In [8]:
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Grant Count by Agency', 'Total Funding by Agency'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}]]
)

fig.add_trace(
    go.Bar(x=grants_by_agency['agency_name'][:10], y=grants_by_agency['grant_count'][:10], 
           name='Grant Count', marker_color='lightblue'),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=grants_by_agency['agency_name'][:10], y=grants_by_agency['total_funding'][:10],
           name='Total Funding', marker_color='darkblue'),
    row=1, col=2
)

fig.update_layout(height=500, showlegend=False, title_text="Grant Analysis by Agency")
fig.update_xaxes(tickangle=45)
fig.show()

In [9]:
print("FUNDING CATEGORIES ANALYSIS")


query_funding_categories = """
SELECT 
    opportunity_category,
    COUNT(*) as grant_count,
    AVG(award_ceiling) as avg_award,
    MAX(award_ceiling) as max_award,
    SUM(estimated_total_program_funding) as total_available
FROM grants
WHERE opportunity_category IS NOT NULL
GROUP BY opportunity_category
ORDER BY grant_count DESC
LIMIT 15
"""

funding_categories = db.execute_query(query_funding_categories)
funding_categories



FUNDING CATEGORIES ANALYSIS


,opportunity_category,grant_count,avg_award,max_award,total_available
0,Discretionary,70123,5.329733e+06,1.397000e+10,1.054163e+12
1,Other,2264,1.003145e+07,2.970000e+09,2.909028e+10
2,Continuation,1535,1.135841e+06,7.000000e+07,5.664413e+09
3,Mandatory,1267,9.107596e+06,3.638000e+08,1.529856e+11
4,Earmark,451,1.964280e+07,1.300000e+09,5.320594e+09


In [10]:
# Visualization - Treemap
fig = px.treemap(
    funding_categories,
    path=['opportunity_category'],
    values='grant_count',
    color='avg_award',
    title='Grant Categories: Size=Count, Color=Avg Award',
    color_continuous_scale='Viridis'
)
fig.update_layout(height=600)
fig.show()

### Non Profits Analysis

In [11]:
print("NONPROFIT DISTRIBUTION BY STATE")


query_by_state = """
SELECT 
    STATE,
    COUNT(*) as org_count,
    AVG(INCOME_AMT) as avg_income,
    AVG(ASSET_AMT) as avg_assets,
    SUM(REVENUE_AMT) as total_revenue
FROM non_profits
WHERE STATE IS NOT NULL AND STATE != ''
GROUP BY STATE
ORDER BY org_count DESC
LIMIT 20
"""

by_state = db.execute_query(query_by_state)
by_state


NONPROFIT DISTRIBUTION BY STATE


,STATE,org_count,avg_income,avg_assets,total_revenue
0,NY,105528,4.620027e+06,6.652602e+06,2.635386e+11
1,NJ,44095,2.001937e+06,2.959992e+06,5.030490e+10
2,MA,38263,8.969090e+06,1.081032e+07,1.515235e+11
3,CT,20943,3.440794e+06,7.130752e+06,3.672936e+10
4,ME,9326,2.070599e+06,2.822893e+06,1.343410e+10
5,NH,8330,1.806223e+06,3.641581e+06,1.079840e+10
6,RI,8113,3.315049e+06,4.668836e+06,1.253819e+10
7,VT,5983,1.573570e+06,2.352664e+06,6.553087e+09
8,IL,1,0.000000e+00,7.500000e+07,0.000000e+00
9,FL,1,1.200000e+08,9.500000e+07,1.000000e+08


In [12]:
# Visualization - Choropleth Map
fig = px.choropleth(
    by_state,
    locations='STATE',
    locationmode="USA-states",
    color='org_count',
    scope="usa",
    title='Nonprofit Organizations by State',
    color_continuous_scale='YlOrRd',
    labels={'org_count': 'Organization Count'}
)
fig.update_layout(height=500)
fig.show()

### FInancial Profile Analysis

In [13]:
print("FINANCIAL PROFILE BY CLASSIFICATION")


query_by_classification = """
SELECT 
    CLASSIFICATION,
    COUNT(*) as org_count,
    AVG(INCOME_AMT) as avg_income,
    AVG(ASSET_AMT) as avg_assets,
    AVG(REVENUE_AMT) as avg_revenue
FROM non_profits
WHERE CLASSIFICATION IS NOT NULL
GROUP BY CLASSIFICATION
HAVING org_count >= 50
ORDER BY avg_income DESC
"""

by_classification = db.execute_query(query_by_classification)
by_classification


FINANCIAL PROFILE BY CLASSIFICATION


,CLASSIFICATION,org_count,avg_income,avg_assets,avg_revenue
0,1800.0,108,1.979450e+07,1.613855e+07,1.922405e+07
1,8000.0,218,1.731015e+07,2.248555e+07,1.911738e+07
2,1280.0,400,1.164431e+07,2.654664e+07,1.093642e+07
3,2000.0,33075,7.507429e+06,1.123642e+07,3.177194e+06
4,1700.0,3042,6.413879e+06,7.371757e+06,6.196569e+06
5,2800.0,537,6.399395e+06,1.419976e+07,4.891250e+06
6,1270.0,1032,6.026498e+06,7.680235e+06,6.047194e+06
7,1000.0,131230,4.322735e+06,6.013848e+06,3.669945e+06
8,0.0,188,3.783318e+06,1.121364e+07,2.678715e+06
9,1200.0,19581,2.929021e+06,4.659651e+06,2.127947e+06


In [14]:
# Visualization - Scatter plot
fig = px.scatter(
    by_classification,
    x='avg_income',
    y='avg_assets',
    size='org_count',
    hover_data=['CLASSIFICATION'],
    title='Financial Profile: Income vs Assets by Classification',
    labels={'avg_income': 'Average Income ($)', 'avg_assets': 'Average Assets ($)'},
    color='org_count',
    size_max=60
)
fig.update_layout(height=600)
fig.show()

### Impact Score Analysis

In [15]:
print("IMPACT SCORE DISTRIBUTION (Integer Bins)")


score_dist = """
SELECT
    impact_score_numeric AS score,
    COUNT(*) AS org_count
FROM non_profits_final
WHERE impact_score_numeric IS NOT NULL
GROUP BY impact_score_numeric
ORDER BY score;
"""

impact_bins = db.execute_query(score_dist)
impact_bins




IMPACT SCORE DISTRIBUTION (Integer Bins)


,score,org_count
0,1,189168
1,2,33602
2,3,17815


In [16]:
# --- Visualization 1: Bar Chart of Score Bins ---
fig = px.bar(
    impact_bins,
    x='score',
    y='org_count',
    title='Impact Score Distribution',
    labels={'score_bin': 'Impact Score Bin', 'org_count': 'Number of Organizations'}
)
fig.update_layout(height=450)
fig.show()


# --- Visualization 2: Pie Chart ---
fig = px.pie(
    impact_bins,
    values='org_count',
    names='score',
    title='Impact Score Distribution (Pie Chart)'
)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(height=450)
fig.show()

In [17]:
print("TOP PERFORMING NONPROFITS")


query_top_performers = """
SELECT 
    nf.NAME,
    nf.STATE,
    nf.CITY,
    nf.CLASSIFICATION,
    nf.impact_score_numeric,
    nf.financial_metric,
    nf.impact_efficiency,
    nf.INCOME_AMT,
    nf.ASSET_AMT
FROM non_profits_final nf
WHERE nf.impact_score_numeric IS NOT NULL
ORDER BY nf.impact_score_numeric DESC
LIMIT 25
"""

top_performers = db.execute_query(query_top_performers)
top_performers

TOP PERFORMING NONPROFITS


,NAME,STATE,CITY,CLASSIFICATION,impact_score_numeric,financial_metric,impact_efficiency,INCOME_AMT,ASSET_AMT
0,MAINE STATE CHAMBER OF COMMERCE,ME,AUGUSTA,1000.0,3,1550130.0,1.935322e-06,1585777.0,709682.0
1,AUGUSTA COUNTRY CLUB,ME,MANCHESTER,1000.0,3,1411076.0,2.126037e-06,1628711.0,1093916.0
2,PORTLAND REGIONAL CHAMBER,ME,PORTLAND,3000.0,3,1070308.0,2.802931e-06,1301751.0,779639.0
3,EASTERN MAINE ELECTRIC COOPERATIVE INC,ME,CALAIS,1000.0,3,15821318.0,1.896176e-07,15821318.0,39372256.0
4,CPORT CREDIT UNION,ME,PORTLAND,1000.0,3,8384031.0,3.578231e-07,8384415.0,173898500.0
5,INTERNATIONAL BROTHERHOOD OF TEAMSTERS,ME,S PORTLAND,3000.0,3,2260275.0,1.327272e-06,2261189.0,334997.0
6,MAINE EDUCATION ASSOCIATION,ME,AUGUSTA,3000.0,3,8431656.0,3.558020e-07,8503978.0,3640881.0
7,BRIDGTON HOSPITAL,ME,LEWISTON,1000.0,3,54148524.0,5.540317e-08,57422296.0,56726536.0
8,PORTLAND COUNTRY CLUB,ME,FALMOUTH,1000.0,3,4000912.0,7.498290e-07,5974176.0,10535130.0
9,PROUTS NECK COUNTRY CLUB,ME,SCARBOROUGH,1000.0,3,1749104.0,1.715164e-06,1774288.0,2900400.0


### Data Reporting Quality Analysis

In [18]:
print("\n4.1 DATA QUALITY OVERVIEW")
print("-" * 70)

# --- SQL: Clear data quality summary ---
query_quality = """
SELECT 
    data_quality,
    COUNT(*) AS org_count,
    ROUND(AVG(confidence_score), 3) AS avg_confidence,

    AVG(CASE WHEN has_mission = 1 THEN 1 ELSE 0 END) * 100 AS pct_has_mission,
    AVG(CASE WHEN has_financial = 1 THEN 1 ELSE 0 END) * 100 AS pct_has_financial,
    AVG(CASE WHEN has_impact = 1 THEN 1 ELSE 0 END) * 100 AS pct_has_impact
FROM nonprofit_quality
GROUP BY data_quality
"""

quality_overview = db.execute_query(query_quality)
print(quality_overview.to_string(index=False))



4.1 DATA QUALITY OVERVIEW
----------------------------------------------------------------------
data_quality  org_count  avg_confidence  pct_has_mission  pct_has_financial  pct_has_impact
   excellent     114851           0.968            100.0              100.0           100.0
        good     125734           0.659            100.0                0.0           100.0


In [19]:

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=[
        'Data Quality Distribution',
        'Completeness by Quality Level'
    ],
    specs=[[{'type': 'pie'}, {'type': 'bar'}]]
)

# --- Pie chart: Count of orgs by quality ---
fig.add_trace(
    go.Pie(
        labels=quality_overview['data_quality'],
        values=quality_overview['org_count'],
        textinfo='percent+label'
    ),
    row=1, col=1
)

# --- Bar chart: % missing vs available fields ---
fig.add_trace(
    go.Bar(
        x=quality_overview['data_quality'],
        y=quality_overview['pct_has_mission'],
        name='Has Mission (%)'
    ),
    row=1, col=2
)

fig.add_trace(
    go.Bar(
        x=quality_overview['data_quality'],
        y=quality_overview['pct_has_financial'],
        name='Has Financial (%)'
    ),
    row=1, col=2
)

fig.add_trace(
    go.Bar(
        x=quality_overview['data_quality'],
        y=quality_overview['pct_has_impact'],
        name='Has Impact (%)'
    ),
    row=1, col=2
)

fig.update_layout(
    title='Data Quality Analysis',
    height=500,
    barmode='group'
)

fig.show()


### Anomaly Summary

In [20]:
print("ANOMALY SUMMARY")

query_anomalies = """
SELECT 
    anomaly_type,
    risk_level,
    COUNT(*) as anomaly_count,
    AVG(anomaly_score) as avg_score,
    MIN(anomaly_score) as min_score,
    MAX(anomaly_score) as max_score
FROM nonprofit_anomalies
WHERE is_anomalous = 1
GROUP BY anomaly_type, risk_level
ORDER BY anomaly_count DESC
"""

anomalies = db.execute_query(query_anomalies)
anomalies


ANOMALY SUMMARY


,anomaly_type,risk_level,anomaly_count,avg_score,min_score,max_score
0,extreme_revenue,Critical,145,5.784566,1.502566,114.335396
1,extreme_revenue,Low,137,0.330667,0.189370,0.495410
2,extreme_revenue,Medium,114,0.696154,0.504451,0.989328
3,extreme_revenue,High,49,1.195237,1.000758,1.451039


In [21]:
# Visualization
fig = px.bar(
    anomalies,
    x='anomaly_type',
    y='anomaly_count',
    color='risk_level',
    title='Detected Anomalies by Type and Risk Level',
    labels={'anomaly_count': 'Count', 'anomaly_type': 'Anomaly Type'}
)

fig.update_layout(
    height=500,
    barmode='group',      
)

fig.show()


In [22]:
print("HIGH-RISK ORGANIZATIONS")


query_high_risk = """
SELECT 
    nf.NAME,
    nf.STATE,
    nf.impact_score_numeric,
    na.anomaly_type,
    na.risk_level,
    na.anomaly_score,
    nf.INCOME_AMT,
    nf.ASSET_AMT
FROM non_profits_final nf
INNER JOIN nonprofit_anomalies na ON nf.EIN = na.EIN
WHERE na.is_anomalous = 1 AND na.risk_level = 'High'
ORDER BY na.anomaly_score DESC
LIMIT 20
"""

high_risk = db.execute_query(query_high_risk)
high_risk

HIGH-RISK ORGANIZATIONS


,NAME,STATE,impact_score_numeric,anomaly_type,risk_level,anomaly_score,INCOME_AMT,ASSET_AMT
0,RENSSELAER POLYTECHNIC INSTITUTE,NY,3,extreme_revenue,High,1.451039,706252900.0,1.430642e+09
1,VASSAR BROTHERS MEDICAL CENTER,NY,3,extreme_revenue,High,1.443848,535821020.0,5.807204e+08
2,MERCY HOSPITAL,ME,3,extreme_revenue,High,1.437792,534298000.0,1.866744e+08
3,HOFSTRA UNIVERSITY,NY,3,extreme_revenue,High,1.410691,681101250.0,1.023155e+09
4,STAMFORD HOSPITAL,CT,3,extreme_revenue,High,1.380581,542516350.0,9.059648e+08
5,COMMUNITY HOSPITAL GROUP INC,NJ,3,extreme_revenue,High,1.380427,523755780.0,2.851666e+08
6,MONMOUTH MEDICAL CENTER INC,NJ,3,extreme_revenue,High,1.366339,515212400.0,6.107683e+08
7,ENGLEWOOD HOSPITAL AND MEDICAL CENTER A NEW JE...,NJ,3,extreme_revenue,High,1.337036,505365600.0,4.892807e+08
8,MARY IMOGENE BASSETT HOSPITAL,NY,3,extreme_revenue,High,1.324001,515784830.0,3.949322e+08
9,BRIDGEPORT HOSPITAL,CT,3,extreme_revenue,High,1.313640,517899260.0,5.321669e+08


### Insights

In [23]:
print("\nIMPACT VS FINANCIAL HEALTH (HIGH QUALITY ORGS ONLY)")


query_impact_financial = """
SELECT 
    nf.NAME,
    nf.STATE,
    nf.CLASSIFICATION,
    nf.impact_score_numeric,
    nf.financial_metric,
    nf.impact_efficiency,
    nf.INCOME_AMT,
    nq.confidence_score,
    nq.data_quality
FROM non_profits_final nf
INNER JOIN nonprofit_quality nq ON nf.EIN = nq.EIN
WHERE nf.impact_score_numeric IS NOT NULL
  AND nq.data_quality = 'excellent'
  AND nf.INCOME_AMT IS NOT NULL
ORDER BY nf.impact_score_numeric DESC
LIMIT 100
"""

impact_financial = db.execute_query(query_impact_financial)
print(f"Found {len(impact_financial)} high-quality organizations")
print(impact_financial.head(15).to_string(index=False))

# Visualization - Scatter plot
fig = px.scatter(
    impact_financial,
    x='financial_metric',
    y='impact_score_numeric',
    size='INCOME_AMT',
    color='CLASSIFICATION',
    hover_data=['NAME', 'STATE'],
    title='Impact Score vs Financial Health (High Quality Orgs)',
    labels={
        'financial_metric': 'Financial Health Metric',
        'impact_score_numeric': 'Impact Score'
    }
)
fig.update_layout(height=600)
fig.show()


IMPACT VS FINANCIAL HEALTH (HIGH QUALITY ORGS ONLY)
Found 100 high-quality organizations
                                  NAME STATE  CLASSIFICATION  impact_score_numeric  financial_metric  impact_efficiency  INCOME_AMT  confidence_score data_quality
       MAINE STATE CHAMBER OF COMMERCE    ME          1000.0                     3         1550130.0       1.935322e-06   1585777.0               0.9    excellent
                  AUGUSTA COUNTRY CLUB    ME          1000.0                     3         1411076.0       2.126037e-06   1628711.0               0.9    excellent
             PORTLAND REGIONAL CHAMBER    ME          3000.0                     3         1070308.0       2.802931e-06   1301751.0               0.9    excellent
EASTERN MAINE ELECTRIC COOPERATIVE INC    ME          1000.0                     3        15821318.0       1.896176e-07  15821318.0               0.9    excellent
                    CPORT CREDIT UNION    ME          1000.0                     3         8384

In [24]:
print("STATE-LEVEL PERFORMANCE ANALYSIS")


query_state_performance = """
SELECT 
    nf.STATE,
    COUNT(*) as org_count,
    AVG(nf.impact_score_numeric) as avg_impact,
    AVG(nf.financial_metric) as avg_financial,
    AVG(nq.confidence_score) as avg_data_quality,
    SUM(CASE WHEN na.is_anomalous = 1 THEN 1 ELSE 0 END) as anomaly_count,
    ROUND(SUM(CASE WHEN na.is_anomalous = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as anomaly_rate
FROM non_profits_final nf
LEFT JOIN nonprofit_quality nq ON nf.EIN = nq.EIN
LEFT JOIN nonprofit_anomalies na ON nf.EIN = na.EIN
WHERE nf.impact_score_numeric IS NOT NULL
GROUP BY nf.STATE
HAVING org_count >= 20
ORDER BY avg_impact DESC
LIMIT 25
"""

state_performance = db.execute_query(query_state_performance)
print(state_performance.to_string(index=False))

# Visualization - Bubble chart
fig = px.scatter(
    state_performance,
    x='avg_financial',
    y='avg_impact',
    size='org_count',
    color='anomaly_rate',
    hover_data=['STATE', 'avg_data_quality'],
    title='State Performance: Impact vs Financial Health',
    labels={
        'avg_financial': 'Average Financial Health',
        'avg_impact': 'Average Impact Score',
        'anomaly_rate': 'Anomaly Rate (%)'
    },
    color_continuous_scale='RdYlGn_r'
)
fig.update_layout(height=600)
fig.show()

STATE-LEVEL PERFORMANCE ANALYSIS
STATE  org_count  avg_impact  avg_financial  avg_data_quality  anomaly_count  anomaly_rate
   MA      38263    1.327549   3.961629e+06          0.828984             92          0.24
   VT       5983    1.308541   1.095688e+06          0.790022              4          0.07
   NY     105528    1.299124   2.497966e+06          0.807161            218          0.21
   CT      20943    1.292842   1.755325e+06          0.797297             36          0.17
   ME       9326    1.278469   1.441353e+06          0.779981             14          0.15
   NH       8330    1.263866   1.296552e+06          0.794034             13          0.16
   RI       8113    1.237890   1.547348e+06          0.836226             16          0.20
   NJ      44095    1.236421   1.140963e+06          0.794142             52          0.12


### Key Metrics

In [30]:
total_nonprofits = db.execute_query("SELECT COUNT(*) as cnt FROM non_profits").iloc[0]['cnt']
total_grants = db.execute_query("SELECT COUNT(*) as cnt FROM grants").iloc[0]['cnt']
avg_impact = db.execute_query("SELECT AVG(impact_score_numeric) as avg FROM non_profits_final WHERE impact_score_numeric IS NOT NULL").iloc[0]['avg']
total_anomalies = db.execute_query("SELECT COUNT(*) as cnt FROM nonprofit_anomalies WHERE is_anomalous = 1").iloc[0]['cnt']
total_funding = db.execute_query("SELECT SUM(estimated_total_program_funding) as total FROM grants WHERE estimated_total_program_funding IS NOT NULL").iloc[0]['total']
high_impact_count = db.execute_query("SELECT COUNT(*) as cnt FROM non_profits_final WHERE impact_score_numeric = 3").iloc[0]['cnt']

print("\n KEY METRICS")
print("-" * 70)
print(f"Total Nonprofits:        {total_nonprofits}")
print(f"Total Grants Available:  {total_grants}")
print(f"Average Impact Score:    {avg_impact:.2f}/10")
print(f"High Impact Orgs (8+):   {high_impact_count}")
print(f"Anomalies Detected:      {total_anomalies}")
print(f"Total Funding Pool:      ${total_funding/1e9} B")


 KEY METRICS
----------------------------------------------------------------------
Total Nonprofits:        240585
Total Grants Available:  75640
Average Impact Score:    1.29/10
High Impact Orgs (8+):   17815
Anomalies Detected:      445
Total Funding Pool:      $1247.223687878 B
